In [1]:
# Dependencias
import os
import math
import tempfile
import tensorflow as tf 
import numpy as np 
import pandas as pd
import sklearn.metrics
import matplotlib.pyplot as plt 
import tensorflow.keras.backend as K
from tensorflow.keras import datasets, utils, preprocessing
from tensorflow.keras import models, losses, optimizers
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from numpy.random import seed
from tensorflow import random

In [2]:
# Comprobar versión de TensorFlow
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

#apaño para un error
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

 

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
#fin apaño

# Fijar semilla
seed(22)
random.set_seed(22)

2.3.0
Num GPUs Available:  0


In [3]:
# Obtener dataset
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data(path="mnist.npz")

In [4]:
# Normalizar datos de test y train (originalmente 255 valores)
x_test = x_test.astype('float32')
x_train = x_train.astype('float32')
x_test /= 255
x_train /= 255

img_rows, img_cols = 28, 28
num_dim = 1
num_classes = 10

# Pasar de matriz a vectores
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, num_dim)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, num_dim)

print(x_test.shape)
print(x_train.shape)

(10000, 28, 28, 1)
(60000, 28, 28, 1)


In [5]:
# Codificación One-hot para las clases
y_test_oh = to_categorical(y_test, num_classes)
y_train_oh = to_categorical(y_train, num_classes)

print(y_test_oh.shape)
print(y_train_oh.shape)

(10000, 10)
(60000, 10)


In [6]:
# Reimplementación en Keras de MNISTNet
# Basada en https://github.com/JJGO/shrinkbench/blob/master/models/mnistnet.py

mnistnet = Sequential()

mnistnet.add(Conv2D(20, 5, input_shape=(img_rows, img_cols, num_dim)))
mnistnet.add(Activation(tf.nn.relu))
mnistnet.add(Conv2D(50, 5))
mnistnet.add(Activation(tf.nn.relu))
mnistnet.add(Flatten())
mnistnet.add(Dense(100))
mnistnet.add(Activation(tf.nn.relu))
mnistnet.add(Dense(y_train_oh.shape[1]))
mnistnet.add(Activation(tf.nn.softmax))


# Imprimimos resumen de la red
mnistnet.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 20)        520       
_________________________________________________________________
activation (Activation)      (None, 24, 24, 20)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 50)        25050     
_________________________________________________________________
activation_1 (Activation)    (None, 20, 20, 50)        0         
_________________________________________________________________
flatten (Flatten)            (None, 20000)             0         
_________________________________________________________________
dense (Dense)                (None, 100)               2000100   
_________________________________________________________________
activation_2 (Activation)    (None, 100)               0

In [7]:
# Definición de hiperparámetros
learning_rate = 0.0001  # learning rate
batch_size = 16   # Tamaño del batch
epochs = 30  # Número de epochs
adam = optimizers.Adam(learning_rate=learning_rate)

# Compilamos la red neuronal
mnistnet.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Entrenamos el modelo
history = mnistnet.fit(x_train, y_train_oh, epochs=epochs, batch_size=batch_size,
                        validation_data=(x_test, y_test_oh))

Epoch 1/30
2344/3750 [=================>............] - ETA: 16s - loss: 0.2166 - accuracy: 0.9382

In [ ]:
# Pintar gráfica de Loss (MSE)
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'b.', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Guardar a disco modelo de Keras en formato h5 y h5py
mnistnet.save('mnistnetKeras.h5')